# News Sites Data Collection

1. start with NPR
2. explore feasability of collecting the following:
    - Headline
    - Author
    - Date
    - Full text
    - Media

1. Get featured article data
2. Get all others

In [126]:
import pandas as pd, requests
from bs4 import BeautifulSoup, SoupStrainer
def npr_headlines_scraper():
    page = requests.get("https://www.npr.org/sections/news/")
    def strain(name, attrs):
        if name == 'div' and dict(attrs).get('class', None) == 'item-info':
            return True
    def teaser_split(header):
        time_teaser = header.select('p.teaser')[0].text
        split = time_teaser.index('•')
        return time_teaser[:split-1], time_teaser[split+2:]
    soup = BeautifulSoup(page.content, 'html.parser', parse_only=SoupStrainer(strain))
    headers = soup.find_all('div', class_='item-info')
    data = []
    for header in headers:
        cat = header.select('h3.slug')[0].text
        link = header.select('p.teaser a')[0]['href']
        title = header.select('.title')[0].text
        time, teaser = teaser_split(header)
        data.append({'title':title,
                     'teaser':teaser,
                     'time':time,
                     'category':cat,
                     'link':link,
                     'source': 'NPR'})
    return data

In [119]:
data = npr_headlines_scraper()

In [120]:
df = pd.DataFrame(data)

In [121]:
df

,category,link,teaser,time,title
0,Politics,https://www.npr.org/2019/10/28/774170607/house...,"House Speaker Nancy Pelosi, D-Calif., says the...","October 28, 2019",House Will Vote To Formalize Impeachment Proce...
1,National,https://www.npr.org/2019/10/28/774074138/getty...,"The Getty wildfire erupted overnight, partiall...","October 28, 2019","Getty Fire Threatens Homes, Snarls Traffic In ..."
2,Law,https://www.npr.org/2019/10/28/774249816/admin...,A month after signing an agreement with El Sal...,"October 28, 2019",Administration Extends Temporary Protected Sta...
3,National Security,https://www.npr.org/2019/10/28/774156283/dog-i...,The world got to know the Belgian Malinois a l...,"October 28, 2019",Dog Injured In Syrian Terrorist Raid Is Hailed...
4,Sports,https://www.npr.org/2019/10/28/774044200/lock-...,Baseball fans booed President Trump in Washing...,"October 28, 2019",'Lock Him Up': Trump Greeted With Boos And Jee...
5,Sports,https://www.npr.org/2019/10/28/774167356/vlatk...,The 43-year-old takes the helm after a success...,"October 28, 2019",Vlatko Andonovski Is Chosen To Coach U.S. Wome...
6,Energy,https://www.npr.org/2019/10/28/774053444/fact-...,The president is renewing his push for U.S. co...,"October 28, 2019",FACT CHECK: President Trump's Plans For Syrian...
7,Shots - Health News,https://www.npr.org/sections/health-shots/2019...,Rapid genome decoding and analysis have made i...,"October 28, 2019",Fast DNA Sequencing Can Offer Diagnostic Clues...
8,Europe,https://www.npr.org/2019/10/28/774031088/rain-...,Days before the Oct. 31 deadline to leave the ...,"October 28, 2019",Rain Chexit: European Union Grants Brexit Dela...
9,Politics,https://www.npr.org/2019/10/28/771287237/trump...,House Democrats are investigating President Tr...,"October 28, 2019",Trump Impeachment Inquiry: A Guide To Key Peop...
